In [26]:

from sqlalchemy import create_engine, MetaData
import urllib.parse
import pyodbc
import pandas as pd

In [27]:
server = "34.196.61.171"
database = "tsbci_staging"
username = "tsbci_readonly"
password = "Sij31o#akus!uc"

In [28]:
connection_string = ("Driver={SQL Server};" 
    "Server=" + server +";" 
    "Database=" + database +";" 
    "UID=" + username + ";" 
    "PWD=" + password)

In [29]:
url = urllib.parse.quote(connection_string)
engine = create_engine("mssql+pyodbc:///?odbc_connect=" + url, use_setinputsizes=False)


In [30]:
mask_columns = {
    'dbo.AspNetUsers': ['PasswordHash', 'SecurityStamp', 'Ein'],
    'dbo.FinancialInstitutions': ['Ein', 'Tins']
}

try:
    connection = engine.connect()
    metadata = MetaData()
    metadata.reflect(bind=engine, schema="dbo")

    table_names = metadata.tables.keys()
    df_dict = dict()
    for count, table in enumerate(table_names):
        print(count)
        print(table)
        df = pd.DataFrame()
        df = pd.read_sql(f"SELECT * FROM {table}", connection)
        print(df.columns.tolist())
        
        if table in mask_columns:
            for column in mask_columns[table]:
                if column in df.columns:
                    df[column] = df[column].apply(lambda x: 'masked' if x is not None and x != '' else x)
        df_dict[table] = df

    with pd.ExcelWriter('tsbci_db_export.xlsx', engine='xlsxwriter') as writer:
        for sheet_name, df in df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name[4:], index=False)

except Exception as e:
    print(f"Error: {e}")

finally:
    if connection:
        connection.close()

0
dbo.__EFMigrationsHistory
['MigrationId', 'ProductVersion']
1
dbo.ApplicationStatusAuditTrail
['AuditId', 'FinancialInstitutionId', 'Status_From', 'Status_To', 'CreatedBy', 'DateCreated', 'Action']
2
dbo.ApplicationStatuses
['BorrowerApplicationStatusId', 'ApplicationStatus', 'CreatedBy', 'DateCreated', 'Description']
3
dbo.AspNetRoleClaims
['Id', 'RoleId', 'ClaimType', 'ClaimValue']
4
dbo.AspNetRoles
['Id', 'Name', 'NormalizedName', 'ConcurrencyStamp']
5
dbo.AspNetUserClaims
['Id', 'UserId', 'ClaimType', 'ClaimValue']
6
dbo.AspNetUsers
['Id', 'UserName', 'NormalizedUserName', 'Email', 'NormalizedEmail', 'EmailConfirmed', 'PasswordHash', 'SecurityStamp', 'ConcurrencyStamp', 'PhoneNumber', 'PhoneNumberConfirmed', 'TwoFactorEnabled', 'LockoutEnd', 'LockoutEnabled', 'AccessFailedCount', 'Approved', 'Discriminator', 'FinancialInstitutionId', 'FirstName', 'LastName', 'Title', 'EnteredInstitution', 'RegistrationDate', 'PrimaryContact', 'BusinessName', 'Ein', 'ExecutiveOfficeAddress', 'Mail